# 6章

In [1]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path(".") / ".env"
load_dotenv(dotenv_path=env_path)  # 環境変数を読み込む


True

In [2]:
# ドキュメントは別リポジトリで管理されている

# from langchain_community.document_loaders import GitLoader


# def file_filter(file_path: str) -> bool:
#     return file_path.endswith(".mdx")A


# loader = GitLoader(
#     clone_url="https://github.com/langchain-ai/langchain",
#     repo_path="./langchain",
#     branch="master",
#     file_filter=file_filter,
# )

# documents = loader.load()
# print(len(documents))


### データベース化対象

In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/docs",
    repo_path="./docs",
    branch="main",
    file_filter=file_filter,
)

documents = loader.load()
print("Loaded documents:", len(documents))


Loaded documents: 1979


### ベクトル化

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import random
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# ----------------------
# 2. サンプリングで件数削減
# ----------------------
sample_fraction = 0.25
sampled_documents = random.sample(documents, k=int(len(documents) * sample_fraction))
print("Sampled documents:", len(sampled_documents))

# ----------------------
# 3. チャンク化
# ----------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,  # 1チャンクあたり最大4000文字
    chunk_overlap=100,  # 200文字重複で文脈をつなぐ
)

split_docs = text_splitter.split_documents(sampled_documents)
print("Split into chunks:", len(split_docs))

Sampled documents: 494
Split into chunks: 1174


In [6]:
# ----------------------
# 4. Embeddings 作成 & Chroma登録
# ----------------------
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma(embedding_function=embeddings, persist_directory="./chroma_db")

In [7]:
from tqdm.notebook import tqdm  # ←Notebook用

batch_size = 100
for i in tqdm(range(0, len(split_docs), batch_size)):
    batch = split_docs[i : i + batch_size]
    db.add_documents(batch)

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("LangChainの概要を教えて")


'LangChainは、言語モデルを活用したアプリケーションを開発するためのフレームワークです。ユーザーは、LangChainのインターフェース標準に従ったコンポーネントを実装するPythonパッケージを作成することができます。これには、チャットモデル、ツール、リトリーバーなどのさまざまなコンポーネントが含まれます。LangChainは、これらのコンポーネントを使用して、言語モデルを効果的に活用するための機能を提供します。'